In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from artifact_detector_model import MARKER_NAMES
from dataset import embed_data_dir
import matplotlib
plt.rcParams["font.family"] = "Serif"

In [ ]:
def addlabels(x, y, s):
    for xi, yi, si in zip(x, y, s):
        plt.text(xi, yi + 0.02, si, ha="center")


f, ax = plt.subplots(figsize=(12, 3))
df_filtered = pd.read_csv("predicted_all_embed.csv")[
    ["image_path"]
    + MARKER_NAMES
]
# image_path here is the absolute path, in the other df it's the relative path.
# make them compatible
df_filtered['image_path'] = df_filtered['image_path'].apply(lambda x: x.replace("/vol/biomedic3/data/EMBED/images/png/1024x768" + '/',''))
df_filtered['image_path'] = df_filtered['image_path'].apply(lambda x: x.replace('/data2/mb121/EMBED/images/png/1024x768' + '/',''))
df_filtered['image_path'] = df_filtered['image_path'].apply(lambda x: x.replace('/data/EMBED/images/png/1024x768' + '/',''))
df_cancer = pd.read_csv(
    "/vol/biomedic3/data/EMBED/tables/mammo-net-csv/embed-non-negative.csv"
)
df_filtered = df_filtered.merge(df_cancer)

neg_percentages, pos_percentages = [], []
marker_labels = [
    "No Artefacts",
    "Circular Marker",
    "Triangular Marker",
    "Breast Implant",
    "Device",
    "Compression",
]

positive = df_filtered[df_filtered["is_positive"] == 1]
negative = df_filtered[df_filtered["is_positive"] == 0]
pos_nothing = positive[positive[MARKER_NAMES].sum(axis=1) == 0]
neg_nothing = negative[negative[MARKER_NAMES].sum(axis=1) == 0]

pos_count = []
neg_count = []
for marker in MARKER_NAMES:
    pos_marker_set = positive[positive[marker] == 1]
    neg_marker_set = negative[negative[marker] == 1]
    pos_percentages.append(len(pos_marker_set) / len(positive))
    neg_percentages.append(len(neg_marker_set) / len(negative))
    pos_count.append(len(pos_marker_set))
    neg_count.append(len(neg_marker_set))

pos_marker_set = pos_nothing
neg_marker_set = neg_nothing
pos_percentages.insert(0, len(pos_marker_set) / len(positive))
neg_percentages.insert(0, len(neg_marker_set) / len(negative))
pos_count.insert(0, len(pos_marker_set))
neg_count.insert(0, len(neg_marker_set))
cmap = matplotlib.colormaps['Blues'].resampled(6)
ax.bar(
    np.arange(len(marker_labels)) - 0.225,
    neg_percentages,
    width=0.45,  # width of the bar
    edgecolor="grey",  # edgecolor of the bar
    color=cmap(6),  # fill color of the bar
    capsize=5,
    label="Normal images (BIRADS 1)",
)
ax.set_ylabel("Proportion of images with artefact")
addlabels(
    np.arange(len(marker_labels)) - 0.24,
    neg_percentages,
    [f"{perc*100:.1f}%\nN={c}" for perc, c in zip(neg_percentages, neg_count)],
)

ax.bar(
    np.arange(len(marker_labels)) + 0.225,
    pos_percentages,
    width=0.45,  # width of the bar
    edgecolor="grey",  # edgecolor of the bar
    color=cmap(1),  # fill color of the bar
    capsize=5,
    label="Suspicious or benign lesions",
)
addlabels(
    np.arange(len(marker_labels)) + 0.24,
    pos_percentages,
    [f"{perc*100:.1f}%\nN={c}" for perc, c in zip(pos_percentages, pos_count)],
)

ax.legend()
ax.set_xticks(np.arange(len(marker_labels)))
ax.set_xticklabels(marker_labels)
ax.set_ylim((0, 1.1))
plt.savefig('output/predicted_stat.pdf', bbox_inches='tight')

In [ ]:

predicted = df_filtered.loc[(df_filtered['compression'] == 1)]
print(len(predicted))
predicted = predicted['image_path'].sample(6).values
print(len(predicted))
import matplotlib
import os
from skimage.io import imread

f, axes = plt.subplots(1, 6, figsize=(25, 10))

for i, ax in enumerate(axes.flat):
    if i >= len(predicted):
        ax.axis("off")
    else:
        test_image = os.path.join(embed_data_dir, predicted[i])
        img_path = os.path.join(test_image)
        image = imread(img_path).astype(np.float32)
        image = (image - np.min(image)) / (np.max(image) - np.min(image))
        image = (image * 255).astype(np.uint8)
        ax.imshow(image, cmap=matplotlib.cm.gray)
        ax.axis("off")
        ax.set_title(i)

plt.subplots_adjust(wspace=0, hspace=0)
plt.show()